In [2]:
import pandas as pd
import jieba
from typing import List, Tuple

In [2]:
def load_excel(file_path: str) -> pd.DataFrame:
    """
    读取Excel文件，验证必要列并处理空值
    要求文件必须包含"商品ID"和"商品名称"列（可修改required_cols适配实际列名）
    """
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        required_cols = ['商品ID', '商品名称']
        # 检查必要列是否存在
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"缺少必要列：{', '.join(missing_cols)}，需包含{required_cols}")
        # 去除商品ID或名称为空的无效行
        df = df.dropna(subset=required_cols).reset_index(drop=True)
        # 强制转为字符串类型，避免数字ID/名称拼接出错（如科学计数法、格式丢失）
        df['商品ID'] = df['商品ID'].astype(str).str.strip()
        df['商品名称'] = df['商品名称'].astype(str).str.strip()
        return df
    except Exception as e:
        print(f"读取文件{file_path}失败：{str(e)}")
        raise

In [3]:
owner_df = load_excel("./美团-快驿点特价超市(虹桥店)全量商品信息20251109.xlsx")
owner_df.head()

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,想买人数,点赞数,最小订购数,标识,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid
0,22639130381,【规格可选】30W充电套装 适用苹果8-14系列PD快充充电头 苹果手机数据线充电器 1个,单独 1米线,6978410500134,6.75,6.75,NaN,3,所搜商品,所搜商品,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,直插充电器,41430057383
1,22638543544,农夫山泉 饮用纯净水 550ml12瓶_包,550ml*12,6921168560424,9.90,22.00,NaN,34,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,包装饮用水,41429308607
2,22636514960,红牛Red Bull 维生素风味饮料 250ml_罐,250ml*1罐,6970640429988,8.07,8.23,NaN,7,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,运动功能饮料（非健字号）,41430928019
3,22640202093,百岁山 饮用天然矿泉水 570ml_瓶,NaN,06922255466476,2.99,2.99,NaN,6,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,天然矿泉水,41431324722
4,22740184269,康师傅茉莉蜜茶调味茶饮品 1000ml_瓶,1000ml*1瓶,6920459991985,5.54,5.65,NaN,1,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,茶饮料,41669396232


In [4]:
ele_df = load_excel("./饿了么-京东便利店（虹桥中心店）全量商品数据20251110.xlsx")
ele_df.head()

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,想买人数,点赞数,最小订购数,标识,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid
0,950657120093,汤达人 日式豚骨拉面 83g(面饼55g+配料28g)/桶,NaN,6925303770563,3.49,8.3,4.3折,100,泡面速食卤蛋火腿,泡面拌面,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,950657120093
1,950103769331,康师傅 BIG大食桶老坛酸菜牛肉面 159g/桶,NaN,6920152424285,4.90,6.5,7.6折,200,泡面速食卤蛋火腿,康师傅面,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,950103769331
2,949473714726,上好佳 鲜虾片膨化食品 80克/袋,NaN,6926265313386,5.20,8.0,6.5折,100,膨化食品干吃脆面,薯条薯片,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,949473714726
3,949474714550,百威 9.7°P啤酒 500ml/听,NaN,6948960100078,0.01,9.5,0.1折,300,酒酒酒,精酿啤酒,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,949474714550
4,949480010428,统一茄皇 茄皇牛肉面 128g/桶,NaN,6925303796426,5.50,7.5,7.4折,75,泡面速食卤蛋火腿,康师傅面,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,949480010428


In [3]:
def chinese_tokenize(text: str) -> set:
    """中文精确分词，返回去重后的分词集合（适配Jaccard相似度计算）"""
    if not text or text.isspace():
        return set()
    return set(jieba.cut(text, cut_all=False))  # 精确模式分词，适合商品名称匹配


def jaccard_similarity(set1: set, set2: set) -> float:
    """计算Jaccard相似度：交集大小 / 并集大小（取值0-1，越大越相似）"""
    if not set1 and not set2:
        return 1.0  # 两个空字符串视为完全相似
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0.0


In [4]:
def find_top3_similar_combined(product_name: str, df2: pd.DataFrame, df2_tokens: List[set]) -> Tuple[str, str, str]:
    """
    找到附件2中Top3相似商品，返回3个"ID-名称"组合字符串（不足3个补空）
    返回格式：(相似商品1, 相似商品2, 相似商品3)，示例：("id1-名称1", "id2-名称2", "id3-名称3")
    """
    target_tokens = chinese_tokenize(product_name)
    similarity_results = []

    # 计算附件2所有商品的相似度（复用预处理的分词结果，提升效率）
    for idx in range(len(df2)):
        candidate_id = df2.iloc[idx]['商品ID']
        candidate_name = df2.iloc[idx]['商品名称']
        candidate_tokens = df2_tokens[idx]
        similarity = jaccard_similarity(target_tokens, candidate_tokens)
        # 存储（负相似度：用于升序排序等价于相似度降序，商品ID-名称组合）
        similarity_results.append((-similarity, f"{candidate_id}-{candidate_name}--{similarity}"))

    # 按相似度降序排序（相同相似度保留附件2原始顺序）
    similarity_results.sort()
    # 提取Top3组合，不足3个时用空字符串填充
    top3_combined = [result[1] for result in similarity_results[:3]]
    while len(top3_combined) < 3:
        top3_combined.append("")  # 补空确保始终返回3个元素

    return top3_combined[0], top3_combined[1], top3_combined[2]

In [5]:
def preprocess_df2_tokens(df2: pd.DataFrame) -> List[set]:
    """预处理附件2的商品名称分词结果，避免重复分词（大幅提升批量处理效率）"""
    return [chinese_tokenize(name) for name in df2['商品名称']]

In [6]:
owner_df=pd.read_csv("./meituan_sku.csv")
owner_df.head()

,商品ID,商品名称,规格,upc码,折扣价,原价,skuid
0,20318304358,（规格自选强力布基胶带网格双面胶高粘度透明无痕地毯地垫沙发垫固定贴 一卷,5cm*10m#2cm*10m,6.902051e+12,12.45,15.0,35112245909
1,20593235769,海氏海诺 一次性医用外科口罩 10只_盒,10只/盒,6.925924e+12,12.96,15.8,35816960188
2,20317711803,爱斐堡 牛奶味生吐司 散装 约70g_袋,70g*1袋,6.971985e+12,4.58,5.2,35111173396
3,20318114339,【1条】雀巢 脆脆鲨巧克力味威化饼干 零食营养能量棒,1根,6.901884e+12,3.76,2.0,35113206574
4,20316956858,天明 桉叶糖 22克_盒,22克*1盒,6.901311e+12,2.64,3.0,35110532938


In [11]:
ele_df=pd.read_csv("./elme_sku.csv")
ele_df.head()

,商品ID,商品名称,规格,条码,折扣价,原价,skuid
0,950657120093,汤达人 日式豚骨拉面 83g(面饼55g+配料28g)/桶,NaN,6925303770563,3.49,8.3,950657120093
1,950103769331,康师傅 BIG大食桶老坛酸菜牛肉面 159g/桶,NaN,6920152424285,4.90,6.5,950103769331
2,949473714726,上好佳 鲜虾片膨化食品 80克/袋,NaN,6926265313386,5.20,8.0,949473714726
3,949474714550,百威 9.7°P啤酒 500ml/听,NaN,6948960100078,0.01,9.5,949474714550
4,949480010428,统一茄皇 茄皇牛肉面 128g/桶,NaN,6925303796426,5.50,7.5,949480010428


In [12]:
df2_tokens = preprocess_df2_tokens(ele_df)


In [13]:
name="味全 每日C 100%纯橙汁 300ml_瓶 （新老包装随机）"
find_top3_similar_combined(name,ele_df,df2_tokens)

('949477802059-味全 每日C 100%纯橙汁 300ml/瓶 （新老包装随机）--0.875',
 '950658772111-味全 每日c葡萄汁饮料 300ml/瓶（新老包装随机）--0.5263157894736842',
 '949014663803-味全每日C 100%投入苹果汁 300ml/瓶--0.3888888888888889')

In [14]:
top1_list = []
top2_list = []
top3_list = []
output_path = "附件1_补充Top3相似商品（ID-名称组合）.xlsx"

for idx, row in owner_df.iterrows():
    # 每处理100个商品显示进度（便于跟踪大量数据）
    if (idx + 1) % 100 == 0:
        print(f"已处理 {idx + 1}/{len(owner_df)} 个商品")

    product_name = row['商品名称']
    # 获取3个"ID-名称"组合（分别对应Top1、Top2、Top3）
    top1, top2, top3 = find_top3_similar_combined(product_name, ele_df, df2_tokens)
    top1_list.append(top1)
    top2_list.append(top2)
    top3_list.append(top3)

owner_df['相似商品1（ID-名称）'] = top1_list  # 第三列：最相似
owner_df['相似商品2（ID-名称）'] = top2_list  # 第四列：次相似
owner_df['相似商品3（ID-名称）'] = top3_list  # 第五列：第三相似

# 5. 保存结果（不覆盖原文件）
owner_df.to_excel(output_path, index=False, engine='openpyxl')
print(f"\n处理完成！结果已保存到：{output_path}")
print("列说明：")
print(" - 第三列：相似商品1（ID-名称）→ 最相似商品")
print(" - 第四列：相似商品2（ID-名称）→ 次相似商品")
print(" - 第五列：相似商品3（ID-名称）→ 第三相似商品")
print("注：不足3个相似商品时，对应列为空字符串")

已处理 100/6288 个商品
已处理 200/6288 个商品
已处理 300/6288 个商品
已处理 400/6288 个商品
已处理 500/6288 个商品
已处理 600/6288 个商品
已处理 700/6288 个商品
已处理 800/6288 个商品


KeyboardInterrupt: 